In [ ]:
import sys
import os

import tensorflow as tf
import tensorflow.contrib.eager as tfe
import keras
from keras.datasets import cifar10

sys.path.append('E:/xinlib')
import xin
import TF
import chaos

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

# 载入数据集

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
print('训练样本数：{}'.format(len(x_train)))
print('测试样本数：{}'.format(len(x_test)))

训练样本数：50000
测试样本数：10000


In [53]:
def transform(x, y):
    return tf.to_float(x), tf.to_int32(y)


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 128
train_dataset = train_dataset.shuffle(
    buffer_size=100000)  # buffer_size 需要大于样本数
train_dataset = train_dataset.batch(batch_size).map(map_func=transform)

# View a single example entry from a batch
features, label = tfe.Iterator(train_dataset).next()
print("example features:", features[0])
print("example label:", label[0])

example features: tf.Tensor(
[[[  93.   85.   61.]
  [ 184.  180.  159.]
  [ 239.  237.  216.]
  ..., 
  [ 190.  188.  164.]
  [ 170.  172.  152.]
  [ 116.  116.  100.]]

 [[ 179.  168.  147.]
  [ 250.  242.  210.]
  [ 238.  231.  205.]
  ..., 
  [  80.   83.   71.]
  [  85.   90.   75.]
  [ 156.  158.  143.]]

 [[ 218.  208.  176.]
  [ 206.  190.  138.]
  [ 202.  185.  141.]
  ..., 
  [ 117.  124.  113.]
  [ 130.  139.  118.]
  [ 192.  197.  173.]]

 ..., 
 [[ 140.  137.  120.]
  [  30.   34.   27.]
  [  24.   35.   18.]
  ..., 
  [  52.   67.   33.]
  [  80.  118.   45.]
  [ 161.  195.  119.]]

 [[ 102.   94.   80.]
  [  65.   66.   55.]
  [  69.   77.   54.]
  ..., 
  [ 100.  113.   79.]
  [ 121.  148.   89.]
  [ 158.  182.  122.]]

 [[  75.   67.   54.]
  [ 139.  138.  121.]
  [ 195.  200.  165.]
  ..., 
  [ 201.  209.  175.]
  [ 192.  204.  163.]
  [ 132.  142.  104.]]], shape=(32, 32, 3), dtype=float32)
example label: tf.Tensor([5], shape=(1,), dtype=int32)


# 创建模型—— `tf.layers`

In [69]:
def model(inputs):
    x1 = tf.keras.layers.Conv2D(filters=20, kernel_size=5, activation='relu')(inputs)
    x2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x1)
    x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, activation='relu')(x2)
    x4 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x3)
    x5 = tf.keras.layers.Flatten()(x4)
    x6 = tf.keras.layers.Dense(128, activation="relu")(x5)
    y_ = tf.keras.layers.Dense(10)(x6)
    return y_

def loss(y, y_):
    return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

def forward(model):
    def fn(inputs, targets):
        y_ = model(inputs)
        loss_value = loss(targets, y_)
        return tfe.gradients_function(loss_value)
    return fn

In [70]:
grad = tfe.gradients_function(forward)

In [73]:
grad(2, 4)

TypeError: forward() takes 1 positional argument but 2 were given

# 训练模型

In [55]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

In [56]:
# Note: Rerunning this cell uses the same model variables

# keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
    epoch_loss_avg = tfe.metrics.Mean()
    epoch_accuracy = tfe.metrics.Accuracy()

    # Training loop - using batches of 32
    for x, y in tfe.Iterator(train_dataset):
        # Optimize the model
        grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.variables),
                                  global_step=tf.train.get_or_create_global_step())

        # Track progress
        epoch_loss_avg(loss(model, x, y))  # add current batch loss
        # compare predicted label to actual label
        epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)

    # end epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                    epoch_loss_avg.result(),
                                                                    epoch_accuracy.result()))

AttributeError: 'function' object has no attribute 'variables'

# `tf.estimator`

# `tf.data.Dataset`

# `tf.metrics`

# `tf.feature_column`